In [166]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time
from selenium.webdriver.common.action_chains import ActionChains
from bs4 import BeautifulSoup
from selenium.webdriver.chrome.options import Options
import requests
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import Select
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException
import os
import csv

In [120]:
# Enable verbose logging
options = webdriver.ChromeOptions()
options.add_argument("--verbose")

driver = webdriver.Chrome(options=options)
driver.get('https://mim-guide.com/specializations')

# Pause to allow page elements to load
time.sleep(5)

# Select 'United States' from the 'Area' dropdown
area_dropdown = driver.find_element(By.ID, 'region-selection')
area_select = Select(area_dropdown)
area_select.select_by_visible_text('United States')

# Wait for the specialization dropdown to be updated based on the area selection
time.sleep(5)

# Select Business Analytics from the 'Specialization' dropdown
spec_dropdown = driver.find_element(By.ID, 'specialization-selection')
spec_select = Select(spec_dropdown)
spec_select.select_by_visible_text('Business Analytics')


In [121]:
# Initialize the page number
page_number = 1
save_directory = os.path.join('Homework', 'DDR', 'Final Project')
# Start the loop
while True:
    # Define the full path to the file
    file_path = os.path.join(save_directory, f"page_{page_number}.html")
    
    # Pause before saving the page
    time.sleep(5)
    
    # Save the current page's HTML to the specified file path
    with open(file_path, "w", encoding="utf-8") as file:
        file.write(driver.page_source)
        print(f"Saved: {file_path}")
    
    # Pause before finding the 'next' button
    time.sleep(5)
    
    try:
        # Find the 'next page' button by its 'rel' attribute
        next_page_button = driver.find_element(By.CSS_SELECTOR, "a[rel='next']")

        # Scroll the button into view and try clicking
        driver.execute_script("arguments[0].scrollIntoView(true);", next_page_button)
        
        # Pause before clicking the 'next' button
        time.sleep(5)
        
        next_page_button.click()
        
        # Pause to wait for the next page to load
        time.sleep(5)
        
        # Increment the page number for the next iteration of the loop
        page_number += 1

    except NoSuchElementException:
        print("Next page button does not exist. Exiting the loop.")
        break
    except ElementClickInterceptedException:
        print("Next page button was not clickable. There might be an overlay or the button is obscured.")
        break

# Close the driver
driver.quit()

Saved: Homework/DDR/Final Project/page_1.html
Saved: Homework/DDR/Final Project/page_2.html
Saved: Homework/DDR/Final Project/page_3.html
Saved: Homework/DDR/Final Project/page_4.html
Saved: Homework/DDR/Final Project/page_5.html
Saved: Homework/DDR/Final Project/page_6.html
Saved: Homework/DDR/Final Project/page_7.html
Saved: Homework/DDR/Final Project/page_8.html
Saved: Homework/DDR/Final Project/page_9.html
Next page button does not exist. Exiting the loop.


In [164]:
schools_info = []  # List to store information about each school

for filename in sorted(os.listdir(save_directory)):  # Sort to ensure processing order
    if filename.endswith(".html"):
        filepath = os.path.join(save_directory, filename)
        with open(filepath, 'r', encoding='utf-8') as file:
            html = file.read()

        print(f"Read {filename}.")

        # Process the HTML content
        soup = BeautifulSoup(html, 'html.parser')

        # Find all 'title' divs
        titles = soup.select('div.title')

        for title in titles:
            school_info = {}  # Dictionary to store info for one school

            a_tag = title.find('a', class_='text-title')
            span = title.find('span', class_='number-title')
            if a_tag and span:
                school_info['name'] = a_tag.get_text(strip=True)
                school_info['rank'] = span.get_text(strip=True)

                item_inner_div = title.find_parent('div', class_='item-inner')
                featured_program_div = item_inner_div.find('div', class_='featured-program')

                if featured_program_div:
                    spans = featured_program_div.find_all('span')
                    if len(spans) >= 3:
                        school_info['length'] = spans[1].get_text(strip=True)
                        school_info['tuition'] = spans[2].get_text(strip=True)
                    else:
                        school_info['length'] = "N/A"
                        school_info['tuition'] = "N/A"
                else:
                    school_info['length'] = "N/A"
                    school_info['tuition'] = "N/A"

                schools_info.append(school_info)
            else:
                print(f"No 'title' div found in {filename}.")



Read page_1.html.
Read page_2.html.
Read page_3.html.
Read page_4.html.
Read page_5.html.
Read page_6.html.
Read page_7.html.
Read page_8.html.
Read page_9.html.


In [165]:
# Optional: Print the collected data to verify
for info in schools_info:
    print(info)

{'name': 'New York University (NYU) - Stern School of Business', 'rank': '1', 'length': 'N/A', 'tuition': 'N/A'}
{'name': 'University of Southern California (USC) - Marshall School of Business', 'rank': '2', 'length': 'N/A', 'tuition': 'N/A'}
{'name': 'MIT Sloan School of Management', 'rank': '3', 'length': '9 months', 'tuition': '65,750'}
{'name': 'Carnegie Mellon University (CMU) - Tepper School of Business', 'rank': '4', 'length': 'N/A', 'tuition': 'N/A'}
{'name': 'University of Michigan - Ross School of Business', 'rank': '5', 'length': '10 months', 'tuition': '64,063'}
{'name': 'Cornell University - Johnson Graduate School of Management', 'rank': '6', 'length': '12 months', 'tuition': '29,500'}
{'name': "Northeastern University - D'Amore-McKim School of Business", 'rank': '7', 'length': 'N/A', 'tuition': 'N/A'}
{'name': 'Pace University - Lubin School of Business', 'rank': '8', 'length': 'N/A', 'tuition': 'N/A'}
{'name': 'University of California, Los Angeles (UCLA) - Anderson Sch

In [167]:
# Specify the CSV file name
csv_file_path = os.path.join(save_directory, 'schools_info.csv')

# Writing to CSV
with open(csv_file_path, mode='w', newline='', encoding='utf-8') as file:
    # Assuming all dictionaries in schools_info have the same structure
    fieldnames = ['name', 'rank', 'length', 'tuition']  # Define the column headers
    writer = csv.DictWriter(file, fieldnames=fieldnames)

    writer.writeheader()  # Write the column headers

    for school in schools_info:
        writer.writerow(school)  # Write each school's information as a row

print(f"Data saved to {csv_file_path}.")

Data saved to Homework/DDR/Final Project/schools_info.csv.
